In [6]:
import os
import json
from datetime import datetime
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from openai import OpenAI
import base64
from email.mime.text import MIMEText
from dotenv import load_dotenv

#### Set up OAuth 2.0 for Calendar + Gmail

In [7]:
load_dotenv()  # Load variables from .env file

SCOPES = os.getenv("SCOPES").split(",")
CLIENT_SECRET_FILE = os.getenv("CLIENT_SECRET_FILE")
API_NAME = os.getenv("API_NAME")
API_VERSION = os.getenv("API_VERSION")



In [8]:
def authenticate_google_services():
    """Authenticate and return both calendar and gmail service clients"""
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
    creds = flow.run_local_server(port=0)
    calendar_service = build("calendar", "v3", credentials=creds)
    gmail_service = build("gmail", "v1", credentials=creds)
    return calendar_service, gmail_service

In [9]:
calendar_service, gmail_service = authenticate_google_services()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=161842291169-u489f31dcc8vlb8vcn5si009jhkj554a.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A61928%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=YspNWiPl6wl2UTFA4ry0soNhOeGb9L&access_type=offline


#### Gmail Send Function

In [10]:
def create_message(to, subject, message_text):
    message = MIMEText(message_text)
    message["to"] = to
    message["subject"] = subject
    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
    return {"raw": raw}

In [11]:
def send_confirmation_email(to_list, subject, message_text):
    for recipient in to_list:
        try:
            message = create_message(recipient, subject, message_text)
            sent_message = (
                gmail_service.users()
                .messages()
                .send(userId="me", body=message)
                .execute()
            )
            print(
                f"✅ Confirmation email sent to {recipient} (Message ID: {sent_message['id']})"
            )
        except Exception as e:
            print(f"❌ Failed to send email to {recipient}: {e}")

#### OpenAI Setup

In [12]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [13]:

tools = [
    {
        "type": "function",
        "function": {
            "name": "create_event",
            "description": "Creates a new event in the calendar with provided details.",
            "parameters": {
                "type": "object",
                "properties": {
                    "summary": {"type": "string"},
                    "location": {"type": "string"},
                    "description": {"type": "string"},
                    "start_time": {"type": "string", "format": "date-time"},
                    "end_time": {"type": "string", "format": "date-time"},
                    "attendees": {
                        "type": "array",
                        "items": {"type": "string", "format": "email"},
                    },
                },
                "required": ["summary", "start_time", "end_time", "attendees"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "update_event",
            "description": "Updates an existing event in the calendar.",
            "parameters": {
                "type": "object",
                "properties": {
                    "event_id": {"type": "string"},
                    "summary": {"type": "string"},
                    "location": {"type": "string"},
                    "description": {"type": "string"},
                    "start_time": {"type": "string"},
                    "end_time": {"type": "string"},
                    "attendees": {
                        "type": "array",
                        "items": {"type": "string", "format": "email"},
                    },
                },
                "required": ["event_id"],
            },
        },
    },
]

#### GPT Prompt + Event Execution

In [14]:
today = datetime.now()
date_context = f"Today is {today.strftime('%A, %B %d, %Y')}."


In [26]:
messages = [
    {
        "role": "system",
        "content": "You are an event manager to create or update an event.",
    },
    {
        "role": "user",
        "content": f"{date_context} Schedule a team meeting tomorrow at 2 PM for 1 hour with muhammadkhubaib182012@gmail.com and pineconelerarner@gmail.com about the new project on google meet, cloud migration.",
    },
]

In [27]:
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)


In [28]:
tool_calls = completion.choices[0].message.tool_calls

#### Event Functions

In [34]:

def create_event(summary, location, description, start_time, end_time, attendees):
    """Create a new event and send confirmation email"""
    event = {
        "summary": summary,
        "location": location,
        "description": description,
        "start": {
            "dateTime": start_time,
            "timeZone": "Asia/Karachi",
        },
        "end": {
            "dateTime": end_time,
            "timeZone": "Asia/Karachi",
        },
        "attendees": [{"email": email} for email in attendees],
        "reminders": {
            "useDefault": False,
            "overrides": [
                {"method": "popup", "minutes": 10},
            ],
        },
        "conferenceData": {
            "createRequest": {
                "requestId": f"meet-{int(datetime.utcnow().timestamp())}",  # unique ID
                "conferenceSolutionKey": {"type": "hangoutsMeet"},
            }
        },
    }

    try:
        created_event = (
            calendar_service.events()
            .insert(
                calendarId="primary",
                body=event,
                conferenceDataVersion=1,  # Required to generate Meet links
            )
            .execute()
        )
        print(f"📅 Event created: {created_event['htmlLink']}")

        meet_link = created_event.get("hangoutLink", "No Google Meet link available.")

        subject = f"Meeting Confirmation: {summary}"
        body = (
            f"You are invited to the meeting:\n\n"
            f"📍 Location: {location}\n"
            f"🗓 Start: {start_time}\n"
            f"🕒 End: {end_time}\n"
            f"📄 Description: {description}\n"
            f"🔗 Event Page: {created_event['htmlLink']}\n"
            f"📞 Google Meet Link: {meet_link}"
        )
        send_confirmation_email(attendees, subject, body)

        return created_event
    except HttpError as error:
        print(f"An error occurred: {error}")
        return None


In [35]:
def update_event(
    event_id,
    summary=None,
    location=None,
    description=None,
    start_time=None,
    end_time=None,
    attendees=None,
):
    try:
        event = (
            calendar_service.events()
            .get(calendarId="primary", eventId=event_id)
            .execute()
        )

        if summary:
            event["summary"] = summary
        if location:
            event["location"] = location
        if description:
            event["description"] = description
        if start_time:
            event["start"]["dateTime"] = start_time
        if end_time:
            event["end"]["dateTime"] = end_time
        if attendees:
            event["attendees"] = [{"email": email} for email in attendees]

        updated_event = (
            calendar_service.events()
            .update(calendarId="primary", eventId=event_id, body=event)
            .execute()
        )
        print(f"🛠️ Event updated: {updated_event['htmlLink']}")
        return updated_event
    except HttpError as error:
        print(f"An error occurred: {error}")
        return None


#### Run Tool Function

In [36]:
def call_function(name, args):
    if name == "create_event":
        return create_event(**args)
    elif name == "update_event":
        return update_event(**args)

In [37]:
for tool_call in tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(completion.choices[0].message)

    result = call_function(name, args)
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
    )


📅 Event created: https://www.google.com/calendar/event?eid=OTFoZXA5NTE4dnM2NXNmbWRsNWxnaW51MDggbWtodWJhaWliMUBt
✅ Confirmation email sent to muhammadkhubaib182012@gmail.com (Message ID: 196a293b89896566)
✅ Confirmation email sent to pineconelerarner@gmail.com (Message ID: 196a293bdf32db86)
